In [181]:
import json
from helper_synthesis import count_sentences_in_text, xml_to_json

path_examples = '../07 train models/synth/Llama70B/fixed/split_4.json'

with open(path_examples, 'r', encoding="utf-8") as file:
    data_dictionary = json.load(file)

In [182]:
# original = [data_dictionary[d] for d in range(len(data_dictionary)) if data_dictionary[d]["llm_aspect_polarity_in_text_but_not_in_label"] > 0]
# with open("example_original.json", 'w') as file:
#     json.dump(original, file)

# data_dictionary = original.copy()


In [183]:
change_dictionary = data_dictionary.copy()

In [184]:
for i in range(len(data_dictionary)):
    if data_dictionary[i]["llm_aspect_polarity_in_text_but_not_in_label"] > 0:
        incrementFreqSentences = 0
        change = 0
        for k in range(len(data_dictionary[i]["llm_retry_statistic"])):    
            # schauen, ob bei diesem durchgang aspect_polarity_in_text_but_not_in_label grund war
            if data_dictionary[i]["llm_retry_statistic"][k]["llm_aspect_polarity_in_text_but_not_in_label"] > change:
                # schauen, ob auch > 2 sätze
                label = data_dictionary[i]["llm_retry_statistic"][k]["llm_label"]
                text = xml_to_json(data_dictionary[i]["llm_retry_statistic"][k]["llm_prediction_raw"], [tuple(l) for l in label], "", 0, False)["text"]
                if count_sentences_in_text(text) > 1:
                    incrementFreqSentences+=1
                    print("i", i, "incrementor", incrementFreqSentences, "n_retries", len(data_dictionary[i]["llm_retry_statistic"]), "k", k)
                    
                change += 1
            
            change_dictionary[i]["llm_retry_statistic"][k]["llm_more_than_one_sentences"] += incrementFreqSentences
        
        change_dictionary[i]["llm_more_than_one_sentences"] += incrementFreqSentences

i 1238 incrementor 1 n_retries 1 k 0


In [185]:
with open(path_examples, "w", encoding="utf-8") as outfile:
    json.dump(change_dictionary, outfile, ensure_ascii=False)